# Functor

Un Functor es una clase tipo y se define como cualquier tipo de dato al que se le puede aplicar una funcion map

Tenemos un Contructor C[_] y dod tipos A y B, queremos aplicar funciones del tipo C[A] => C[B], asi que necesitamos transformaciones del tipo:

In [7]:
trait Functor[F[_]] {
    def map[A,B](fa: F[A])(f: A => B): F[B]
}

defined trait Functor

![](images/functor2.png)

Podemos testear esto con la clase Option:

In [1]:
sealed abstract class OptionFun[+A] {
    def getValue: A = this match {
        case SomeStuff(x) => x
        case NoneStuff => throw new Exception
    }
    
    def isEmpty: Boolean = this eq NoneStuff
    def map[B](f:A => B): OptionFun[B] = if(isEmpty) NoneStuff else SomeStuff(f(this.getValue))
}

final case class SomeStuff[+A](value:A) extends OptionFun[A]{
    def get: A = value
}

case object NoneStuff extends OptionFun[Nothing]

defined class OptionFun
defined class SomeStuff
defined object NoneStuff

In [16]:
val opt = SomeStuff(15)
println("new value " + opt.map(_.toString).getValue)

new value 15


opt: SomeStuff[Int] = SomeStuff(15)

# Applicative

Applicative es una clase tipo y se define como cualquier tipo de dato al que se le puede aplicar una funcion apply

Apply coge un functor que tiene una funcion en el y otro functor y extrae esa funcion del primer functor y lo mapea con el segundo.

Tenemos un Contructor C[_] y dos tipos A y B,  queremos aplicar funciones del tipo C[A] => C[B], asi que necesitamos transformaciones del tipo:

In [5]:
trait Applicative[F[_]] extends Functor[F] {
    def pure[A](a: A): F[A]
    def apply[A,B](ff: F[A => B])(fa: F[A]): F[B]
    def map[A, B](fa: F[A])(f: A => B): F[B] = apply(pure(f))(fa)
}

defined trait Applicative

![](images/applicative.png)

Usando nuestra clase OptionFun:

In [23]:
object OptionFun {
    def pure[A](x: A): OptionFun[A] = if(x == null) NoneStuff else SomeStuff(x)
}
sealed abstract class OptionFun[+A] {
    def getValue: A = this match {
        case SomeStuff(x) => x
        case NoneStuff => throw new Exception
    }
    
    def isEmpty: Boolean = this eq NoneStuff
    def map[B](f:A => B): OptionFun[B] = if(isEmpty) NoneStuff else SomeStuff(f(this.getValue))
    def apply[B](f: OptionFun[A => B]): OptionFun[B] = f.map(f => f(this.getValue))
}

final case class SomeStuff[+A](value:A) extends OptionFun[A]{
    def get: A = value
}

case object NoneStuff extends OptionFun[Nothing]

defined object OptionFun
defined class OptionFun
defined class SomeStuff
defined object NoneStuff

In [24]:
val a: Int => String = _.toString

a: Int => String = ammonite.$sess.cmd23$Helper$$Lambda$2806/1150605593@4d727083

In [25]:
val functionOptionFun = OptionFun.pure(a)

functionOptionFun: OptionFun[Int => String] = SomeStuff(
  ammonite.$sess.cmd23$Helper$$Lambda$2806/1150605593@4d727083
)

In [27]:
val opt = SomeStuff(15)
println("new value " + opt.apply(functionOptionFun).getValue)

new value 15


opt: SomeStuff[Int] = SomeStuff(15)

Puede parecer que no es muy util pero lo es.

# Monads

Monadas aplican una función que devuleve un valor envuelto de un valor envuelto (complicado de explicar).

![](images/monad.png)

Una monada es una clase tipo y es un tipo de dato que implementa la función flatMap

Tenemos un Constructor C[_] y dos tipos A y B, queremos aplicar funciones del tipo C[A] => C[B],  asi que necesitamos transformaciones del tipo:

In [28]:
trait Monad[F[_]] extends Applicative[F] {
    def flatMap[A,B](ff: A => F[B])(fa: F[A]): F[B]
}

defined trait Monad

Usando nuestra clase OptionFun:

In [29]:
object OptionFun {
    def pure[A](x: A): OptionFun[A] = if(x == null) NoneStuff else SomeStuff(x)
}
sealed abstract class OptionFun[+A] {
    def getValue: A = this match {
        case SomeStuff(x) => x
        case NoneStuff => throw new Exception
    }
    
    def isEmpty: Boolean = this eq NoneStuff
    def map[B](f:A => B): OptionFun[B] = if(isEmpty) NoneStuff else SomeStuff(f(this.getValue))
    def apply[B](f: OptionFun[A => B]): OptionFun[B] = f.map(f => f(this.getValue))
    def flatMap[B](f: A => OptionFun[B]): OptionFun[B] = if(isEmpty) NoneStuff else f(this.getValue)
}

final case class SomeStuff[+A](value:A) extends OptionFun[A]{
    def get: A = value
}

case object NoneStuff extends OptionFun[Nothing]

defined object OptionFun
defined class OptionFun
defined class SomeStuff
defined object NoneStuff

¿Porque es tan importante el método flatMap? Con este método puedes componer funciones con valores envueltos. Vamos a ver un ejemplo:

In [30]:
val a = SomeStuff(5)
val b = SomeStuff(6)
val c = SomeStuff(7)

a: SomeStuff[Int] = SomeStuff(5)
b: SomeStuff[Int] = SomeStuff(6)
c: SomeStuff[Int] = SomeStuff(7)

Si queremos sumar todos los valores, con los métodos map y flatMap podemos:

In [31]:
a.flatMap(value => b.flatMap(value2 => c.map(value3 => value + value2 + value3)))

res30: OptionFun[Int] = SomeStuff(18)

Pero no es muy bonito. Podemos hacerlo de manera mas limpia usando for comprehension:

In [32]:
for {
    value1 <- a
    value2 <- b
    value3 <- c
} yield (value1 + value2 + value3)

res31: OptionFun[Int] = SomeStuff(18)